# Assignment 1

Deadline: 19.03.2025, 12:00 CET

<Add your name, student-id and emal address>
Stephan Krushev, s09-727-090, stephan.krushev@gmail.com

In [1]:
# Import standard libraries
import os
import sys
import timeit # To compute runtimes
from typing import Optional

# Import third-party libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Import local modules
#project_root = os.path.dirname(os.path.dirname(os.getcwd()))   # Change this path if needed

project_root =os.getcwd() # set in .vscode/settings.json
print(project_root)


src_path = os.path.join(project_root, 'qpmwp-course/src')
sys.path.append(project_root)

print(src_path)


/home/steph/Documents/Learning/UZH_Portfolio_Management_Python_FS2025/Code
/home/steph/Documents/Learning/UZH_Portfolio_Management_Python_FS2025/Code/qpmwp-course/src


In [2]:
sys.path.append(src_path)
from estimation.covariance import Covariance
from estimation.expected_return import ExpectedReturn
from optimization.constraints import Constraints
from optimization.optimization import Optimization, Objective
from optimization.optimization_data import OptimizationData
from optimization.quadratic_program import QuadraticProgram, USABLE_SOLVERS
print(USABLE_SOLVERS) # edit in quaratic_program.py to add more solvers or subtract if needed, uncomment mosek and gurobi

{'cvxopt', 'mosek', 'osqp', 'quadprog', 'daqp', 'highs', 'qpalm', 'gurobi'}


## 1. Solver Horse Race

### 1.a)
(3 points)

Generate a Multivariate-Normal random dataset of dimension TxN, T=1000, N=100, and compute a vector of expected returns, q, and a covariance matrix, P, using classes ExpectedReturn and Covariance respectively.

In [3]:

# remove the follwoing solvers if using more than 800 stocks
#USABLE_SOLVERS.remove('quadprog') 
#USABLE_SOLVERS.remove('daqp')
#USABLE_SOLVERS.remove('mosek')
#USABLE_SOLVERS.remove('gurobi')

In [4]:

# Set the dimensions
T = 1000  # Number of time periods
N = 100   # Number of assets

# remove daqp and quadprog solver from the list of usable solvers if not already done

# Generate a random mean vector and covariance matrix for the multivariate normal distribution

# Set the random seed for reproducibility
np.random.seed(0)

# Generate the mean vector and covariance matrix

mean = np.random.randn(N) * 0.01 
random_matrix = np.random.randn(N, N)*0.01
cov = random_matrix @ random_matrix.T # create a positive semi-definite matrix. In practice positive definite, because stocks are not perfectly correlated
# for bigger matrices, use additional approximation for the covariance matrix, e.g. Higham (2002) algorithm or something faster.


# typical daily returns 1%
# typical daily volatility 1%, variance 10^-4



# mean = <your code here>
# cov = <your code here>

# Generate the Multivariate-Normal random dataset
data = np.random.multivariate_normal(mean, cov, size=T)

# Convert the dataset to a DataFrame for easier manipulation
df = pd.DataFrame(data, columns=[f'Asset_{i+1}' for i in range(N)])

# Compute the vector of expected returns (mean returns) from df
#q =  np.mean(data, axis=0)

q = np.exp(np.log(1 + df).mean(axis=0)) - 1

# Compute the covariance matrix from df
P = np.cov(data, rowvar=False)

# Display the results
print("Vector of expected returns (q):") # typical daily returns 1%
print(q)

print("\nCovariance matrix (P):") # typical daily volatility 1%, variance 10^-4
print(P)

Vector of expected returns (q):
Asset_1      0.014602
Asset_2     -0.000493
Asset_3      0.001541
Asset_4      0.020163
Asset_5      0.007384
               ...   
Asset_96     0.001896
Asset_97    -0.005400
Asset_98     0.007971
Asset_99    -0.007407
Asset_100   -0.002762
Length: 100, dtype: float64

Covariance matrix (P):
[[ 0.01029453  0.00080807  0.00018417 ...  0.00169781  0.00012145
  -0.00104138]
 [ 0.00080807  0.00878875  0.0013538  ...  0.00108371  0.00012731
   0.00024923]
 [ 0.00018417  0.0013538   0.00902655 ...  0.00013418  0.00020117
  -0.00123718]
 ...
 [ 0.00169781  0.00108371  0.00013418 ...  0.01206623  0.00048459
   0.00046833]
 [ 0.00012145  0.00012731  0.00020117 ...  0.00048459  0.00886367
   0.00063175]
 [-0.00104138  0.00024923 -0.00123718 ...  0.00046833  0.00063175
   0.00961142]]


### 1.b)
(3 points)

Instantiate a constraints object by injecting column names of the data created in 1.a) as ids and add:
- a budget constaint (i.e., asset weights have to sum to one)
- lower bounds of 0.0 for all assets
- upper bounds of 0.2 for all assets
- group contraints such that the sum of the weights of the first 30 assets is <= 0.3, the sum of assets 31 to 60 is <= 0.4 and the sum of assets 61 to 100 is <= 0.5

In [5]:
# Instantiate the Constraints class
constraints = Constraints(ids = df.columns.tolist())



# Add budget constraint
#<your code here>

#constraints.add_budget_constraint()
constraints.add_budget(rhs=1, sense='=')

# Add box constraints (i.e., lower and upper bounds)
#<your code here>

constraints.add_box(lower=0, upper=0.2)


# Add linear constraints
#<your code here> G*x <= h
G = pd.DataFrame(np.zeros((3, N)), columns=constraints.ids)
G.iloc[0, 0:30] = 1
G.iloc[1, 30:60] = 1
G.iloc[2, 60:N] = 1
h = pd.Series([0.3, 0.4, 0.5])
constraints.add_linear(G=G, sense='<=', rhs=h)

constraints.budget
constraints.box
constraints.linear

{'G':    Asset_1  Asset_2  Asset_3  Asset_4  Asset_5  Asset_6  Asset_7  Asset_8  \
 0      1.0      1.0      1.0      1.0      1.0      1.0      1.0      1.0   
 1      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 2      0.0      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
 
    Asset_9  Asset_10  ...  Asset_91  Asset_92  Asset_93  Asset_94  Asset_95  \
 0      1.0       1.0  ...       0.0       0.0       0.0       0.0       0.0   
 1      0.0       0.0  ...       0.0       0.0       0.0       0.0       0.0   
 2      0.0       0.0  ...       1.0       1.0       1.0       1.0       1.0   
 
    Asset_96  Asset_97  Asset_98  Asset_99  Asset_100  
 0       0.0       0.0       0.0       0.0        0.0  
 1       0.0       0.0       0.0       0.0        0.0  
 2       1.0       1.0       1.0       1.0        1.0  
 
 [3 rows x 100 columns],
 'sense': 0    <=
 dtype: object,
 'rhs': 0    0.3
 1    0.4
 2    0.5
 dtype: float64}

### 1.c) 
(4 points)

Solve a Mean-Variance optimization problem (using coefficients P and q in the objective function) which satisfies the above defined constraints.
Repeat the task for all open-source solvers in qpsolvers and compare the results in terms of:

- runtime
- accuracy: value of the primal problem.
- reliability: are all constarints fulfilled? Extract primal resisduals, dual residuals and duality gap.

Generate a DataFrame with the solvers as column names and the following row index: 'solution_found': bool, 'objective': float, 'primal_residual': float, 'dual_residual': float, 'duality_gap': float, 'runtime': float.

Put NA's for solvers that failed for some reason (e.g., unable to install the package or solvers throws an error during execution). 




In [6]:
# Extract the constraints in the format required by the solver
import time
GhAb = constraints.to_GhAb()

# Loop over solvers, instantiate the quadratic program, solve it and store the results
#<your code here>

risk_aversion = 3
# initialize output dictionary
results = {}

for solver in USABLE_SOLVERS:
    qp = QuadraticProgram(
        P = P * risk_aversion,
        q = q.to_numpy() * -1,
        G = GhAb['G'],
        h = GhAb['h'],
        A = GhAb['A'],
        #b = GhAb['b'],
        b = np.array([GhAb['b']]), # convert to numpy array for gurobi
        lb = constraints.box['lower'].to_numpy(),
        ub = constraints.box['upper'].to_numpy(),
        solver = solver,
    )
    start_time = time.time()
    qp.solve()
    end_time = time.time()
    elapsed_time = end_time - start_time
    solution = qp.results.get('solution')
    print(f"{solver}: {qp.objective_value()}")
    print(f"Elapsed time: {elapsed_time}")
    # store the results in a dict of dicts
    results[solver] = {
        'is_feasible': qp.is_feasible(),
        #'status': solution.status,
        'objective_value': qp.objective_value(),
        #'solution': solution,
        #'solution_time': solution.solve_time,
        'solution_found': solution.found,
        'primal_residual': solution.primal_residual(),
        'dual_residual': solution.dual_residual(),
        'duality_gap': solution.duality_gap(),#[0],
        'elapsed_time': elapsed_time,
    }
    


################################################################
# daqp fails with 800 stocks
################################################################
# print weights

print(solution.x)


cvxopt: -0.012904192899859744
Elapsed time: 0.04244661331176758
mosek: -0.012904197304984526
Elapsed time: 0.05507516860961914
osqp: -0.013107105170067126
Elapsed time: 0.009341716766357422
quadprog: -0.012904197353319132
Elapsed time: 0.007252931594848633
daqp: -0.012904197374840993
Elapsed time: 0.0029532909393310547
highs: -0.012904197353315284
Elapsed time: 0.01625514030456543
qpalm: -0.012911026198373381
Elapsed time: 0.006988525390625
Set parameter Username
Set parameter LicenseID to value 2631927
Academic license - for non-commercial use only - expires 2026-03-06
gurobi: -0.012904197175937618
Elapsed time: 0.060671329498291016
[2.11671662e-09 5.57626627e-11 7.16319777e-11 9.99999654e-02
 9.48470893e-11 3.88309197e-11 2.70984817e-10 5.44351127e-11
 5.01452608e-11 5.93681514e-11 5.93154400e-11 6.77413783e-10
 1.27071960e-10 4.69812270e-11 6.71089075e-11 5.05106854e-11
 2.50801613e-10 4.88353892e-11 5.40967561e-11 4.02756594e-11
 2.25804527e-11 7.69817758e-11 8.06551173e-11 4.11416

Print and visualize the results

In [9]:
#<your code here>
# print the results in a table format
results_df = pd.DataFrame(results).T
results_df


,is_feasible,objective_value,solution_found,primal_residual,dual_residual,duality_gap,elapsed_time
gurobi,True,-0.012904,True,0.0,0.0,0.0,0.050602
qpalm,True,-0.012911,True,0.000019,0.0,0.000007,0.007296
cvxopt,True,-0.012904,True,0.0,0.0,0.0,0.005648
quadprog,True,-0.012904,True,0.0,0.0,0.0,0.006967
osqp,True,-0.013297,True,0.00098,0.000053,0.00038,0.004011
daqp,True,-0.012904,True,0.0,0.0,0.0,0.002189
mosek,True,-0.012904,True,0.0,0.0,0.0,0.045235
highs,True,-0.012904,True,0.0,0.0,0.0,0.01146


## Check the results

In [10]:
# check the sum of the weights is equal to 1
print("sum: ", np.sum(solution.x))
# check no weight is bigger than 0.2
print("max: ", np.max(solution.x))
# check no weight is smaller than 0
print("min: ", np.min(solution.x))
#print sum of weights in each group: 0.3, 0.4, 0.5

print("sum of weights in each group: ", np.dot(G, solution.x))

sum:  1.0000000000000002
max:  0.2
min:  -1.3877787807814457e-17
sum of weights in each group:  [0.3        0.26148634 0.43851366]


## Output for 5000 stocks:

<div>
<style scoped>
    .dataframe tbody tr th:only-of-type {
        vertical-align: middle;
    }

    .dataframe tbody tr th {
        vertical-align: top;
    }

    .dataframe thead th {
        text-align: right;
    }
</style>
<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>is_feasible</th>
      <th>objective_value</th>
      <th>solution_found</th>
      <th>primal_residual</th>
      <th>dual_residual</th>
      <th>duality_gap</th>
      <th>elapsed_time</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>gurobi</th>
      <td>True</td>
      <td>0.080612</td>
      <td>True</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>11.241116</td>
    </tr>
    <tr>
      <th>cvxopt</th>
      <td>True</td>
      <td>0.080612</td>
      <td>True</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>117.19363</td>
    </tr>
    <tr>
      <th>mosek</th>
      <td>True</td>
      <td>0.080614</td>
      <td>True</td>
      <td>0.0</td>
      <td>0.00007</td>
      <td>0.000001</td>
      <td>132.735383</td>
    </tr>
    <tr>
      <th>osqp</th>
      <td>True</td>
      <td>-0.000901</td>
      <td>True</td>
      <td>0.001801</td>
      <td>0.000149</td>
      <td>0.069756</td>
      <td>36.577922</td>
    </tr>
    <tr>
      <th>highs</th>
      <td>True</td>
      <td>0.080612</td>
      <td>True</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>0.0</td>
      <td>6.637514</td>
    </tr>
    <tr>
      <th>qpalm</th>
      <td>True</td>
      <td>0.080079</td>
      <td>True</td>
      <td>0.00007</td>
      <td>0.0</td>
      <td>0.000532</td>
      <td>412.799718</td>
    </tr>
  </tbody>
</table>
</div>

## 2. Analytical Solution to Minimum-Variance Problem

(5 points)

- Create a `MinVariance` class that follows the structure of the `MeanVariance` class.
- Implement the `solve` method in `MinVariance` such that if `solver_name = 'analytical'`, the analytical solution is computed and stored within the object (if such a solution exists). If not, call the `solve` method from the parent class.
- Create a `Constraints` object by injecting the same ids as in part 1.b) and add a budget constraint.
- Instantiate a `MinVariance` object by setting `solver_name = 'analytical'` and passing instances of `Constraints` and `Covariance` as arguments.
- Create an `OptimizationData` object that contains an element `return_series`, which consists of the synthetic data generated in part 1.a).
- Solve the optimization problem using the created `MinVariance` object and compare the results to those obtained in part 1.c).


In [26]:
# Define class MinVariance
class MinVariance(Optimization):

    def __init__(self,
                 constraints: Constraints,
                 covariance: Optional[Covariance] = None,
                 **kwargs):
        super().__init__(
            constraints=constraints,
            **kwargs
        )
        self.covariance = Covariance() if covariance is None else covariance

    def set_objective(self, optimization_data: OptimizationData) -> None:
        #<your code here>
        P = np.array(self.covariance.data)
        # Ensure return series is a numpy array
        q = np.array(optimization_data.return_series)
        
        # Check if P and q are correctly formatted
        if P is None or q is None:
            raise ValueError("P and q must be provided.")
        
        self.objective = Objective(
            P=P,
            q=q
        )

      

    def solve(self) -> None:
        if self.params.get('solver_name') == 'analytical':
            #<your code here>
            Sigma = self.covariance
            ones = np.ones(Sigma.shape[0])
            
            try:
                inv_Sigma = np.linalg.inv(Sigma)
                weights = inv_Sigma @ ones / (ones.T @ inv_Sigma @ ones)
                self.results["weights"] = weights
            except np.linalg.LinAlgError:
                print("Covariance matrix is singular, falling back to numerical solution.")
                return super().solve()
 #           return None
        else:
            return super().solve()


# Create a constraints object with just a budget constraint
#<your code here>
constraints = Constraints(ids = df.columns.tolist())
constraints.add_budget(rhs=1, sense='=')
# Instantiate the MinVariance class
#<your code here>
min_var = MinVariance(constraints=constraints, covariance=cov, solver_name='analytical')
#min_var = MinVariance(constraints=constraints, covariance=cov, solver_name='any')

# Prepare the optimization data and prepare the optimization problem
#<your code here>
optimization_data = OptimizationData(return_series=q)
min_var.set_objective(optimization_data)
# Solve the optimization problem and print the weights
#<your code here>
min_var.solve()
print("Optimal Weights:", min_var.results['weights'])

Optimal Weights: [-0.03973384  0.02556432  0.04168659  0.14378887 -0.07605087 -0.06821528
 -0.04270917  0.0153855   0.02619449 -0.01558827  0.04210866  0.07953279
  0.01983386 -0.0297907  -0.00459884  0.12365363 -0.06110229  0.1034879
 -0.08934887  0.02114829  0.09690074 -0.10895595 -0.04332748 -0.02157568
 -0.01608325 -0.00922257 -0.0224109   0.05655577  0.06312939 -0.03574108
  0.00532272 -0.10366038 -0.08188086  0.0240462   0.10234561  0.02940148
 -0.08968455  0.01855233 -0.07503303 -0.0716256   0.06260822 -0.00307044
 -0.02134275 -0.00189171  0.09503373  0.02120163  0.04589214 -0.03079066
  0.07272494  0.08168427  0.1297631  -0.01454735 -0.03502958 -0.10093688
  0.05285201 -0.09593372  0.04834424 -0.0682054   0.03440751 -0.1256212
  0.00023455  0.02468415 -0.11157215  0.02386584  0.01407934  0.07652248
  0.14208579  0.06849467  0.01625084  0.01424443  0.00213562  0.1159508
 -0.00568389  0.02212842  0.06749733  0.04697547  0.03025332  0.02084003
  0.06405058  0.00334602  0.01765324 

In [34]:
sum(min_var.results['weights'])


np.float64(0.9999999999999999)

In [31]:
min_var1 = MinVariance(constraints=constraints, covariance=cov, solver_name='any')
min_var1.set_objective(optimization_data)
# Solve the optimization problem and print the weights
#<your code here>
min_var1.solve()
print("Optimal Weights:", min_var1.results['weights'])

Optimal Weights: {'Asset_1': -282.17409729693713, 'Asset_2': 241.88832041092022, 'Asset_3': 219.2926108777159, 'Asset_4': 606.795009586757, 'Asset_5': -583.4749227326205, 'Asset_6': 116.27312820949254, 'Asset_7': -268.33729924417605, 'Asset_8': 50.921517314722415, 'Asset_9': 534.0209371112279, 'Asset_10': 15.710382399023256, 'Asset_11': -196.76182474262416, 'Asset_12': 296.04565917449344, 'Asset_13': 522.7313228572432, 'Asset_14': 76.29294915154551, 'Asset_15': 369.9711385087102, 'Asset_16': -131.46445741910446, 'Asset_17': -286.665104324881, 'Asset_18': 182.5602596865112, 'Asset_19': -953.7515921956609, 'Asset_20': 421.33666879102503, 'Asset_21': 93.43958577275079, 'Asset_22': -539.2273920329513, 'Asset_23': -208.39161166492573, 'Asset_24': -10.646630436494439, 'Asset_25': -424.25591954791594, 'Asset_26': -109.61999827404283, 'Asset_27': -307.53990426068935, 'Asset_28': 855.0137629592241, 'Asset_29': 466.3798243385457, 'Asset_30': -302.46560115852634, 'Asset_31': 94.73740960640346, 'A

In [33]:
sum(min_var1.results['weights'].values())


0.9999999989378807